<a href="https://colab.research.google.com/github/nickklos10/SerieA_Machine_Learning_Predictions_2025/blob/main/Serie_A_Managers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

url_market = "https://www.transfermarkt.us/serie-a/erfolgreicheTrainer/pokalwettbewerb/IT1/plus/1/galerie/0?aktiveTrainer=&saisonIdVon=2004&saisonIdBis=2004&anzahl=1&group=1"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36'
}

response = requests.get(url_market, headers=headers)

if response.status_code == 200:
    print("Request successful!")
    html_content = response.text
else:
    print(f"Request failed with status code {response.status_code}")

Request successful!


In [ ]:
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
table = soup.find('table', class_='items')

year_select = soup.find('select', {'name': 'saisonIdVon'})
selected_year = year_select.find('option', selected=True).get('value')

coach_names = []
clubs = []
nationalities = []
matches = []
seasons = []
finals = []
titles = []
points = []
ppm = []
years = []

In [ ]:
for row in table.find_all('tr', class_=['odd', 'even']):
    # Extract the coach name
    coach_tag = row.find('a', title=True)
    if coach_tag:
        coach_name = coach_tag.get('title')
    else:
        coach_name = None
    coach_names.append(coach_name)


    club_td = row.find_all('td', class_='zentriert')
    if len(club_td) > 2 and club_td[2].find('img'):
        club = club_td[2].find('img')['title']
    else:
        club = None
    clubs.append(club)


    if len(club_td) > 1 and club_td[1].find('img'):
        nationality = club_td[1].find('img')['alt']
    else:
        nationality = None
    nationalities.append(nationality)

    if len(club_td) > 3 and club_td[3].find('a'):
        match_count = club_td[3].find('a').text.strip()
    else:
        match_count = None
    matches.append(match_count)

    if len(club_td) > 4:
        season_count = club_td[4].text.strip()
    else:
        season_count = None
    seasons.append(season_count)

    if len(club_td) > 5:
        final_count = club_td[5].text.strip()
    else:
        final_count = None
    finals.append(final_count)

    if len(club_td) > 6:
        title_count = club_td[6].text.strip()
    else:
        title_count = None
    titles.append(title_count)

    if len(club_td) > 7 and club_td[7].find('a'):
        point_count = club_td[7].find('a').text.strip()
    else:
        point_count = None
    points.append(point_count)

    ppm_td = row.find_all('td', class_='zentriert hauptlink')
    if ppm_td and ppm_td[0].find('a'):
        ppm_text = ppm_td[0].find('a').text.strip().replace(',', '.')
        try:
            ppm_value = float(ppm_text)
        except ValueError:
            ppm_value = None
    else:
        ppm_value = None
    ppm.append(ppm_value)

    years.append(selected_year)

In [ ]:
df = pd.DataFrame({
    'Coach': coach_names,
    'Club': clubs,
    'Nationality': nationalities,
    'Matches': matches,
    'Seasons': seasons,
    'Finals': finals,
    'Titles': titles,
    'Points': points,
    'PPM': ppm,
    'Year': years
})

In [ ]:
numeric_columns = ['Matches', 'Seasons', 'Finals', 'Titles', 'Points', 'PPM']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Group by 'Club' and 'Year' to handle multiple coaches per team per year
df_grouped = df.groupby(['Club', 'Year']).agg({
    'Coach': 'first',
    'Nationality': 'first',
    'Matches': 'first',
    'Seasons': 'first',
    'Finals': 'first',
    'Titles': 'first',
    'Points': 'first',
    'PPM': 'mean'
}).reset_index()

In [ ]:
def append_df_to_csv(df, filename):
    if not os.path.isfile(filename):
        df.to_csv(filename, index=False)
        print(f"Created new file and saved data to '{filename}'")
    else:
        df.to_csv(filename, mode='a', header=False, index=False)
        print(f"Appended data to existing file '{filename}'")

original_csv = 'serie_a_coaches.csv'
grouped_csv = 'serie_a_coaches_grouped.csv'

append_df_to_csv(df, original_csv)

append_df_to_csv(df_grouped, grouped_csv)


print("Grouped DataFrame (One Coach per Team with Averaged PPM):")
print(df_grouped)

print("\nOriginal DataFrame:")
print(df)

Appended data to existing file 'serie_a_coaches.csv'
Appended data to existing file 'serie_a_coaches_grouped.csv'
Grouped DataFrame (One Coach per Team with Averaged PPM):
                 Club  Year                Coach     Nationality  Matches  \
0            AC Milan  2004      Carlo Ancelotti           Italy       38   
1      ACF Fiorentina  2004          Sergio Buso           Italy       13   
2         ACR Messina  2004        Bortolo Mutti           Italy       38   
3             AS Roma  2004        Luigi Delneri           Italy       24   
4         Atalanta BC  2004          Delio Rossi           Italy       24   
5     Bologna FC 1909  2004        Carlo Mazzone           Italy       40   
6      Brescia Calcio  2004      Alberto Cavasin           Italy       15   
7     Cagliari Calcio  2004     Daniele Arrigoni           Italy       38   
8       Chievo Verona  2004    Maurizio D'Angelo           Italy        3   
9         Inter Milan  2004      Roberto Mancini          